In [2]:
import pandas
import os
datadir = 'd:\\plant-pathology-2021-fgvc8\\'
csvfile = os.path.join(datadir, 'train.csv')
dataframes = pandas.read_csv(csvfile)
print("File at {} contains {} rows".format(csvfile, len(dataframes)))

File at d:\plant-pathology-2021-fgvc8\train.csv contains 18632 rows
